Root finding and `while` loops


In physics we are often faced with the problem of finding the solution (or solutions) of an algebraic equation of the form $f(x) = 0$. These solutions are called the “roots” of the function $f(x)$. In some cases, we can solve for $x$ analytically. In most cases, we cannot. In this lesson you will learn two numerical techniques for solving equations of the form $f(x) = 0$.

## Learning objectives: 

* Learn to find zeros of functions using two root finders
* Practice using `while` loops

# `for` loops vs. `while` loops 

So far, we've mostly been using `for` loops to repeat the same calculation over and over a fixed number of times, but in today's module, we'll need a way to run *until some condition is satisifed*.  Therefore, we'll switch to another kind of loop: the `while` loop. These take the form

    while someCondition:
        # Do some stuff
        
where `someCondition` is of type `bool` and the loop runs as long as `someCondition` remains true. This is useful when we don't know how often something should run. We may, for example, wish to get 2 quantities, $a$ and $b$ to "match" by being within `0.001` of each other:

    while abs(a-b) > 0.001:
        # move the values a and/or b to new ones
        
There is another way to run this type of loop which can be useful: the `while True` construction.  It looks like this:

    while True:
        # Do some stuff
        
        if someCondition:
            break
            
        # Do some more stuff

The `while` loop's condition is always true, but we've placed the condition in the middle somewhere with a `break` statement.  A `break` statement exits the __innermost__ loop it's in. An advantage of this construction is that you have to worry less about making sure that your condition is true the first time the loop is called.  It also feels somewhat more intuitive since `someCondition` is the thing you want, not the negative of the thing you want (see example below). Figuring out clever ways to use a `while` loop is a creative process!


Below is a (somewhat contrived) example of its usage. Play with it so that you try and figure out what it is doing, and then bring your questions to class.

In [ ]:
import numpy as np # This while loop will run until the value of log(x) > pi
# If log(x) < pi, we will replace x by 2x
# Note that log is the natural log
x = 1.0
# Notice: this runs as long as log(x) < pi 

while np.log(x) < np.pi:  #this has a LESS THAN comparison
    print("x={}, log(x)={}".format(x, np.log(x)))
    x = x * 2.0
    
print("Result:",x, np.log(x))
print("")

# This is the other construction
x = 1.0
while True:
    print("x={}, log(x)={}".format(x, np.log(x)))
    
    # Here is our condition
    if np.log(x) > np.pi: #this has a GREATER THAN comparison
        print("Condition is true! Exiting the loop")
        break # Exit the while loop
        
    x = 2*x
print("Result:",x, np.log(x))

# Root Finding Example

In this exercise, we will predict *when* and *where* a falling object will hit the ground, accounting for drag.


### ODE Math preview: there are analytic solutions to the equations of motion with drag!

Previously, we've solved (numerically) the equations of motion for a particle with drag (valid for low velocities):

$$
\begin{aligned}
\frac{dy}{dt} &= v(t) \\
\frac{dv}{dt} &= -g - \beta v(t)
\end{aligned}
$$

As you take more math classes, you'll discover that these equation can be solved analytically:

$$
y(t) = y_0 - \frac{g t}{\beta} + \frac{v_0 + g/\beta}{\beta} \bigg( 1- \exp(-\beta t)\bigg)
$$

where $y_0$ and $v_0$ are the initial y-position and y-velocity, respectively. 

*Optional side-road:* As you learn more about ODEs, you'll find out that you can check that this $y(t)$ is a solution by calculating $v(t) = \frac{dy}{dt}$ and plugging it back into the ODE. Feel free to try this out if you're curious! We'll also ask Mathematica to do this for us, in Module 12.

*How long is the projectile in the air?* To answer this question, let's find where $y=y_0$, assuming that's the ground level (and that the ground is level). If we let $t_L$ denote the "landing" time, then this means we're want to solve the equation $y(t_L) = y_0$, which gives:

$$
-\frac{g t_L}{\beta} + \frac{v_0 + g/\beta}{\beta} \bigg( 1- \exp(-\beta t_L)\bigg) = 0
$$

As expected, $t_L = 0$ is a solution of this equation: this is our launch point. However, we’re interested in finding a second solution with $t_L > 0$. This equation does have a solution with $t_L > 0$, as we expect from physical reasoning. However, if you try to use algebra to solve the equation, you'll find that it cannot be expressed in terms of the parameters ($g$, $\beta$, and $v_0$) using standard functions like powers, trig functions, logarithms, etc. Therefore, we will use numerical methods to find the landing time $t_L > 0$.

## Exercise: find the root $t_L$ by hand

The equation above has the form $f(t_L) = 0$, and we want to find the landing time $t_L$. First, we will do this by hand, so write (and then plot) a Python function that takes (as input) a time $t$ and computes $y(t)$.

Choose $y_0 = 0$, $\beta = 0.02$/s, $g =$ 9.8 m/s$^2$, and $v_0 =$ 35.0 m/s for the parameter values. See if you can find $t_L$ to within $\pm 0.01$ s by guess–and–check.


In [ ]:
import numpy as np
import pylab as py

def yfunc(t):

    # fill in this function
    y0 = 0
    beta = 0.02
    g = 0.8
    v0 = 35
    yout = y0 - g*t/beta + (v0+g/beta)*(1-np.exp(-beta*t))/beta
    
    return yout

t = np.linspace(0,8,100)
y = yfunc(t)


py.plot(t,y)
py.axhline(0,color='k',ls='--')
py.show()

In [ ]:
print("Guess = ", yfunc(6.8))

## Automated root-finding

Talk with your group about how could you program a computer to systematically search for the solution $t_L$, based on what you tried by hand.

Many of you probably started with an initial guess, then increased (or decreased) your guess in small steps until the function $y(t)$ changed sign. The value of the solution $t_L$ is between the last two guesses. With this method, the accuracy of the answer can be increased by decreasing the step size. However, if you choose a small step size, you are forced to take very many steps (unless the initial guess is really good). Thus, this method of finding a solution is highly inefficient. 

Below, we will try two other methods that are much more efficient: the bisection method and Newton’s method.

## The bisection method

The bisection method is a simple, robust method of root finding. For a continuous function $f(x)$, we start by assuming a root exists between points $x = a$ and $x = b$. If $f(a)$ and $f(b)$ have opposite signs, then there must be at least one root between $a$ and $b$. If $a$ and $b$ are sufficiently close together, then the midpoint $c = (a + b)/2$ is a good approximation to the root. If $a$ and $b$ are not particularly close, then we use the midpoint $c = (a + b)/2$ to replace either $a$ or $b$ (depending on the relative signs of $f(a)$, $f(b)$ and $f(c)$). We repeat this process until $a$ and $b$ are close enough that the root is accurately determined by the midpoint.

We will use a `while` loop to accomplish this. Here is the algorithm:

1. Choose some $a$ and $b$, with $b > a$, such that $f(a)$ and $f(b)$ have opposite signs.
2. Let $c = (a+b)/2$
3. if sign( f(a) ) == sign( f(c) ) then $c$ is smaller than the root and we replace the value of $a$ with $c$
4. if sign( f(b) ) == sign( f(c) ) then $c$ is larger than the root and we replace the value of $b$ with $c$
5. Goto 2 until either $|a-b|<\epsilon$ or $|f(a) - f(b)| < \epsilon$, where $\epsilon$ is some chosen tolerance.

The calculation guarantees that the final average $(a + b)/2$ is within $\pm \epsilon$ of the actual root. The bisection method is robust, meaning it almost always succeeds in finding a root. With each cycle through the step–by–set procedure, the error in the answer is cut in half. This is an improvement over the method described at the end of the last section. Can you see why this is called the “bisection method”?

**One hint: the bisection method works best if you have good guesses for $a$ and $b$** (which you do ...)

## Exercise

Use the bisection method to find the landing time $t_L$ for the projectile with linear drag. Choose $\beta = 0.02/s$, $g = 9.8m/s^2$, and $v_0 = 35.0 m/s$ for the parameter values. Your answer should be accurate to within $\pm 0.001s$. Experiment with different initial values for $a$ and $b$ so you can see how long it takes and what happens if you make a bad guess. Make a plot of at least one of your choices, with the landing time indicated.

Your report should contain:
1. A plot of the projectile with the landing point indicated on it somehow.  You can make a single point plot, or use the `py.text` or `py.annotate` function to add more information.
2. A numerical value for the landing point.

*Hint:* You may wish to make a function for `f(t)`

*Hint:* You can use the `numpy.sign()` function, but make sure you turn it into an integer using `int()` if you're going to use the `==` operator for comparing them.

In [ ]:
# Equation
def yfunc(t):
  return -g*t/beta + (v0+g/beta)*(1-np.exp(-beta*t))/beta

# Initial Conditions
beta = 0.02
g = 9.8
v0 = 35.0

# time and postion arrays for plotting
t = np.linspace(0,8)
y = yfunc(t)

# Upper and Lower Values for t
t_a = 6.0
t_b = 8.0 

# Error Tolerance
errT = 0.001 

# While Loop
while np.abs(t_a-t_b) > errT:
  t_c= (t_a+t_b)/2
  y_a= yfunc(t_a)
  y_b= yfunc(t_b)
  y_c= yfunc(t_c)
  if np.sign(y_c) == np.sign(y_a): # Replacement if (a,c) same sign
    t_a=t_c
  elif np.sign(y_c) == np.sign(y_b): # Replacement if (b,c) same sign
    t_b=t_c
    t_c= (t_a+t_b)/2

# Root between timeL and timeL+1.
print('Root is: ', t_c, 's') 
print('Corr. y= ', y_c, 'm')

# Plot
py.plot(t,y)
py.plot(t_c, y_c, 'ko')
py.text(t_c+0.1, y_c+5, 'root', fontsize=12)
py.axhline(0,color='k',ls='-')
py.title("Projectile with Liner Drag")
py.ylabel('Position (m)')
py.xlabel('Time (s)')
py.show()

## Newton's Method

Newton's method (aka the Newton–Raphson method) is much more efficient than the bisection method — it reaches the answer with a comparable error in fewer steps. However, Newton’s method is less robust than bisection, and it can be more difficult to apply because it requires knowledge of the derivative of $f(x)$. Here's how it works.

Let $x_1$ denote an initial guess for a root of $f(x)$. Consider the Taylor expansion of $f(x)$ about $x_1$:

$$
f(x) = f(x_1) + f'(x_1)(x − x_1 ) + ...
$$

The unwritten terms are proportional to $(x − x1)^2$, $(x − x1)^3$, etc. Let’s assume for the moment that $x$ is the root we’re looking for; that is, $f(x) = 0$. Now, if the guess $x_1$ is close to the actual root $x$, then the unwritten terms will be small. Dropping these terms, the resulting equation becomes

$$
0 = f(x_1) + f'(x_1)(x − x_1)
$$

This can be solved for the root $x$. Note, however, that this equation is only approximate because we have dropped the higher order terms. As a result, this value $x$ will not be the exact root. We can consider $x$ to be our second guess for the root. Instead of $x$, let’s call it $x_2$. Then the result for $x_2$ is

$$
x_2 = x_1 − \frac{f(x_1)}{ f′(x_1)}
$$

In most cases, $x_2$ will be a closer approximation to the root than $x_1$. Clearly we can iterate this process:

$$
x_{i+1} = x_i − \frac{f(x_i)}{ f'(x_i)}
$$

With each iteration, the approximation $x_{i+1}$ should be closer to the actual root of $f(x)$ than the previous approximation $x_i$.

!["Animation"](https://upload.wikimedia.org/wikipedia/commons/e/e0/NewtonIteration_Ani.gif)

## An application: The Rocket Equation


Here is another problem that involves root finding. The equation governing the motion of a rocket moving in one dimension (along the x–axis) in outer space is

$$
\frac{dv}{dt} = −\frac{u}{m} \frac{dm}{dt} 
$$

Here, $v$ is the x—component of the rocket’s velocity, measured with respect to a fixed inertial frame, The mass of the rocket, including the unburned fuel it contains, is denoted $m$. Finally, $u$ is the speed of the exhaust gas relative to the rocket.

Assuming $u$ is constant in time, this can be integrated to give

$$
v(t) = v_0 − u \ln \left( \frac{m(t)}{m_0} \right) 
$$

where $v_0$ and $m_0$ are the velocity and mass at $t = 0$. If the rocket burns fuel at a constant rate, then 

$$
m(t) = m_0 − \rho t,
$$
where $\rho = −\frac{dm}{dt}$ is the rate of fuel consumption. Setting $v = \frac{dx}{dt}$ and integrating again, we find

$$
x(t)=x_0 +(u+v_0)t + u \left(\frac{m_0}{\rho}−t \right) \ln \left( 1− \frac{\rho t}{m_0} \right)
$$
where $x_0$ is the initial position along the x–axis.

## Exercise

A rocket whose initial mass is 5000 kg burns fuel at a rate of 200 kg/s. The relative speed of the exhaust gas is 2000 m/s. Write a computer code that uses Newton's method to determine the time required for the rocket, starting from rest, to travel 4 km. Your answer should be accurate to within 0.001 s. 

*Hint:* You may wish to make functions for `x(t)` and `v(t)` etc, so that they are easy to use.

*Hint:* For Newton's method, what should the equivalent of $f(t)$ be to solve this problem? Here's one wrong answer: it's not $x(t)$, because finding the zero of that yields where $x(t)=0$, which is not quite what we want.

*Hint:* Newton's method may be complicated at first.  You can try it with bisection to get an idea of the answer and to make sure your Newton's method agreed.  You should also be able to tell from the plot whether your answer is right.

Your report should contain:
1. A plot of the rocket with the 4 km point indicated on it somehow. For example: a single point plot, or use the pylab `annotate` function.
2. A numerical value for the time at which the rocket reaches that point, along with the velocity of the rocket, and the mass of the rocket at that time. 
3. What values you used for your starting guess, and the number of iterations it took for Newton's method to find the answer.

In [ ]:
# Initial Conditions
x0 = 0.0 # [m] Initial Position 
v0 = 0.0 # [m/s] Initial Velocity
u = 2000.0 # [m/s] Eshaust Speed
m0 = 5000.0 # [kg] Iniital Mass
rho = 200.0 # [kg/s] Burn Rate

# Define function for x(t)
def pos(t):
  return x0+ (u+v0)*t + u*(m0/rho-t)*np.log(1-rho*t/m0)

# Define function for x'(t) for Newton's methond
def vel(t):
  return v0 - u*np.log(mass(t)/m0)

# Define function for mass
def mass(t):
  return m0-rho*t

# Define Derivative Function for Newton Method
def deriv(t):
  return v0 - u*np.log((m0-rho*t)/m0)


# Time and Position Arrays for Ploting
t = np.linspace(0.0,10)
y = pos(t)

# Initial Guess and Accuracy
acc = 0.001
t_guess = 8

print('Im using', t_guess, 's for my iniital guess.')
# Define Newtons Method
def Newton(t):
  return t - ((pos(t)-4000)/deriv(t))

# Run Counter Variable
c = 0

# While Loop to Check Positon is clse to 4000 and accuracy closer to 0
while abs(4000.0 - pos(t_guess)) > acc:
  # Execcute Newton's Method
  t_guess = Newton(t_guess)
  # Increase Count
  c = c + 1

print('Time for Rocket to fly 4 km is', t_guess, 's')
print('Position is', pos(t_guess), 'm')
print('Corr. speed is', vel(t_guess), 'm/s')
print('Corr. mass is', mass(t_guess), 'kg')
print("It took", c, "steps to find the time at 4 km")


# Plot the output
py.plot(t,y)
py.plot(t_guess, pos(t_guess), 'ko')
py.axhline(4000, color='k', ls='--')
py.ylabel('Position (m)')
py.xlabel('Time (s)')
py.title('Rocket Trajectory')
py.show()
